In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mar Del Plata,AR,2020-08-28 18:40:40,-38.00,-57.56,62.60,55,0,8.05
1,1,Cabo San Lucas,MX,2020-08-28 18:44:45,22.89,-109.91,82.99,69,90,11.41
2,2,Arraial Do Cabo,BR,2020-08-28 18:43:33,-22.97,-42.03,78.80,69,0,23.04
3,3,Filingue,NE,2020-08-28 18:44:45,14.35,3.32,87.73,54,98,7.61
4,4,Dikson,RU,2020-08-28 18:44:45,73.51,80.55,49.57,80,12,10.33


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable. (center is the geographic center of earth and zoom level makes the map smaller)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable. The list comprehension makes negative temps 0, as google heatmaps wont plot negatives
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, 
                                 max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Switching gears to getting the weather wishes from customers so we can recommend a place to go

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Cabo San Lucas,MX,2020-08-28 18:44:45,22.89,-109.91,82.99,69,90,11.41
2,2,Arraial Do Cabo,BR,2020-08-28 18:43:33,-22.97,-42.03,78.80,69,0,23.04
3,3,Filingue,NE,2020-08-28 18:44:45,14.35,3.32,87.73,54,98,7.61
8,8,Karratha,AU,2020-08-28 18:44:46,-20.74,116.85,77.63,28,0,9.15
16,16,Jerez,GT,2020-08-28 18:44:47,14.10,-89.75,78.80,83,40,3.36
32,32,Acarau,BR,2020-08-28 18:44:49,-2.89,-40.12,83.39,60,56,22.01
34,34,Hilo,US,2020-08-28 18:44:50,19.73,-155.09,78.80,65,1,4.70
35,35,Denia,ES,2020-08-28 18:44:50,38.84,0.11,84.00,68,42,21.00
36,36,Hithadhoo,MV,2020-08-28 18:44:50,-0.60,73.08,80.19,68,100,11.03
37,37,Sattahip,TH,2020-08-28 18:44:50,12.67,100.90,77.00,88,20,3.36


In [13]:
# count can see if there are any null values.  if all the column headings have the same count, there are no nulls
preferred_cities_df.count()

City_ID       199
City          199
Country       199
Date          199
Lat           199
Lng           199
Max Temp      199
Humidity      199
Cloudiness    199
Wind Speed    199
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# .copy will copy the preferred cities df, so we can modify without alterning the oirginal
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
# adds a new column to the new df where entries will be strings
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,82.99,22.89,-109.91,
2,Arraial Do Cabo,BR,78.80,-22.97,-42.03,
3,Filingue,NE,87.73,14.35,3.32,
8,Karratha,AU,77.63,-20.74,116.85,
16,Jerez,GT,78.80,14.10,-89.75,
32,Acarau,BR,83.39,-2.89,-40.12,
34,Hilo,US,78.80,19.73,-155.09,
35,Denia,ES,84.00,38.84,0.11,
36,Hithadhoo,MV,80.19,-0.60,73.08,
37,Sattahip,TH,77.00,12.67,100.90,


In [24]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [25]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Cabo San Lucas,MX,82.99,22.89,-109.91,Hotel Tesoro Los Cabos
2,Arraial Do Cabo,BR,78.80,-22.97,-42.03,Pousada Porto Praia
3,Filingue,NE,87.73,14.35,3.32,
8,Karratha,AU,77.63,-20.74,116.85,ibis Styles Karratha
16,Jerez,GT,78.80,14.10,-89.75,
...,...,...,...,...,...,...
550,Bagu,PH,77.38,18.42,121.36,
552,Lhokseumawe,ID,77.95,5.18,97.15,OYO 3278 Penginapan Alfajri
559,Mbini,GQ,76.15,1.57,9.61,
566,Le Vauclin,MQ,89.60,14.55,-60.84,Hôtel Cap Macabou


In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
# adds a pop up description to each marker using html
# dl = descriptive list, dt = descriptive term, dd = descriptive definition.  
#it's like here's a list of info, these are the terms, this is what the terms mean
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
# hotel into is equal to the info box content, and the list comprehension iterates through each row in the hotel df and
# applies the info box format to the information in that row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [31]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))